In [36]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

In [38]:
# n_components = 20
# PCA_model = PCA(n_components=n_components)
# X = PCA_model.fit_transform(X)

# explained_variance = PCA_model.explained_variance_ratio_

# sns.barplot(x=[i for i in range(1, len(explained_variance)+1)], y=explained_variance)
# plt.show()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
minmax_scale = MinMaxScaler((0, 2*np.pi)).fit(X)
X_train = minmax_scale.transform(X_train)
X_test = minmax_scale.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

c:\Users\Abdo\Anaconda3\envs\Quantum\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Abdo\Anaconda3\envs\Quantum\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [40]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test.to_numpy(),  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [41]:
n_qubits = 16
n_layers = 1

dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qlayer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=list(np.arange(n_qubits)), rotation='X')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))    
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

In [42]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = torch.rand(n_layers, n_qubits, 3) * 1e-3
        self.output_layer = nn.Linear(n_qubits, 10)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x)) * self.scale
        batch_size = x.size(0)
        out = torch.empty((batch_size, n_qubits), dtype=torch.float32)
        for batch_index in range(batch_size):
            expval_tensors = qlayer(x[batch_index], self.qnn_weights)
            expval_floats = [t.item() for t in expval_tensors]
            out[batch_index] = torch.tensor(expval_floats)
        x = self.output_layer(out)  
        return x

model = HybridClassifier()

In [45]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.output_layer = nn.Linear(16, 10)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)
        return x

model = HybridClassifier()

In [46]:
learning_rate = 1e-3
batch_size = 32
epochs = 30

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        #X, y = X.to(torch_device), y.to(torch_device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    with torch.no_grad():
        for X, y in dataloader:
           # X, y = X.to(torch_device), y.to(torch_device)
            pred = model(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")
    
    return test_loss

In [47]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training\n--------------------------")
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loss = test_loop(test_loader, model, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop(test_loader, model, loss_fn)
print("Done!")

Before training
--------------------------
Epoch 1
--------------------------
loss: 2.288773 [   32/56000
loss: 1.244923 [ 3232/56000
loss: 0.616738 [ 6432/56000
loss: 0.614512 [ 9632/56000
loss: 0.362705 [12832/56000
loss: 0.602500 [16032/56000
loss: 0.363992 [19232/56000
loss: 0.964508 [22432/56000
loss: 0.208646 [25632/56000
loss: 0.192730 [28832/56000
loss: 0.256801 [32032/56000
loss: 0.255391 [35232/56000
loss: 0.311931 [38432/56000
loss: 0.350276 [41632/56000
loss: 0.076499 [44832/56000
loss: 0.106581 [48032/56000
loss: 0.073957 [51232/56000
loss: 0.103202 [54432/56000
Accuracy: 94.12100219726562
Avg loss: 0.206725 

Epoch 2
--------------------------
loss: 0.169850 [   32/56000
loss: 0.405935 [ 3232/56000
loss: 0.082833 [ 6432/56000
loss: 0.210112 [ 9632/56000
loss: 0.030682 [12832/56000
loss: 0.315814 [16032/56000
loss: 0.027142 [19232/56000
loss: 0.433003 [22432/56000
loss: 0.028945 [25632/56000
loss: 0.048435 [28832/56000
loss: 0.055345 [32032/56000
loss: 0.204920 [35232/5600